# 1、使用FUNCTION_CALL模式

In [5]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = "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"
8
# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)


# 提供提示词模板（以ChatPromptTemplate为例）
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一个乐于助人的ai助手，根据用户的提问，必要时调用Search工具，使用互联网检索数据"),
    ("system","{agent_scratchpad}"),
    ("system","{chat_history}"),  #添加一个chat_history的变量，用于记录上下文的记忆
    ("human","{input}"),
    # ("placeholder","{agent_scratchpad}")
])

# 提供记忆的实例：ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history", #此值需要与提示词模板中的记录记忆的变量同名
)

# 获取Agent的实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询今天北京的天气情况"})

# 处理响应
print(result)


C:\Users\shkst\AppData\Local\Temp\ipykernel_66412\4220173726.py:47: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(




> Entering new AgentExecutor chain...

Invoking: `Search` with `北京天气情况 2023年10月10日`


[{'title': '2023年10月中旬北京天气穿什么 - 抖音', 'url': 'https://www.douyin.com/shipin/7288577161567602725', 'content': 'Image 30: 早间天气预报（2023年10月10日 08:30）#北京 #天气 #多云 00:55早间天气预报（2023年10月10日 08:30）#北京 #天气 #多云 查看AI文稿 AI文稿 \n今天白天，北京天空中的云亮逐渐增多，最高气温在二十二度左右，适宜朋友们户外活动。今天前半夜，天空中的云亮依然较多，半夜前后逐渐转晴，最低气温在十度上下，山区的最低气温在六到九度之间。 到了明天白天，北京大部分时间依然是晴好的天气。从傍晚前后开始，由于转受偏南气流的影响，北京天空中的云亮逐渐增多。 而随着高空槽主体的到来，十二号傍晚到夜间，北京将迎来一次小雨天气过程，后半夜天气逐渐转晴。 十三号的白天，随着冷空气的南下，天气晴好，还伴有二三季的小北风。不过未来三天的气温变化不大，白天的最高气温在二十到二十三度，夜间的最低气温在十到十二度之间。最后提示大家，近一段时间早晚气温较低，朋友们外出请合理调整着装，谨防着凉感冒。    81北京气象 [...] Image 85: 早间天气预报（2023年10月10日 08:30）#北京 #天气 #多云 00:55早间天气预报（2023年10月10日 08:30）#北京 #天气 #多云 查看AI文稿 AI文稿 \n今天白天，北京天空中的云亮逐渐增多，最高气温在二十二度左右，适宜朋友们户外活动。今天前半夜，天空中的云亮依然较多，半夜前后逐渐转晴，最低气温在十度上下，山区的最低气温在六到九度之间。 到了明天白天，北京大部分时间依然是晴好的天气。从傍晚前后开始，由于转受偏南气流的影响，北京天空中的云亮逐渐增多。 而随着高空槽主体的到来，十二号傍晚到夜间，北京将迎来一次小雨天气过程，后半夜天气逐渐转晴。 十三号的白天，随着冷空气的南下，天气晴好，还伴有二三季的小北风。不过未来三天的气温变化不大，白天的最高气温在二十到二十三度，夜间的最低气温在十到十

In [6]:
result = agent_executor.invoke({"input":"上海的呢？"})

print(result)



> Entering new AgentExecutor chain...

Invoking: `Search` with `今天上海的天气情况`


[{'title': '中国气象局-天气预报-城市预报- 上海', 'url': 'https://weather.cma.cn/web/weather/58367.html', 'content': '时间 23:00 02:00 05:00 08:00 11:00 14:00 17:00 20:00\n天气Image 16Image 17Image 18Image 19Image 20Image 21Image 22Image 23\n气温 31.1℃30.8℃30.6℃30.8℃31.1℃30.4℃29.4℃27.6℃\n降水 0.2mm 无降水 无降水 0.5mm 1.2mm 1.8mm 7.3mm 1.2mm\n风速 3.2m/s 3.3m/s 3m/s 3.3m/s 3.3m/s 3.3m/s 2.9m/s 2.7m/s\n风向 东北风 东北风 西北风 西北风 东北风 西北风 西北风 西北风\n气压 1008.9hPa 1007.9hPa 1007.9hPa 1009.5hPa 1009.4hPa 1008.4hPa 1009.3hPa 1010.1hPa\n湿度 81.6%83.8%87.8%84.8%82.4%86%87%79.4%\n云量 99.5%98.7%94.7%95.3%88.1%98.4%94.3%91.7% [...] 时间 08:00 11:00 14:00 17:00 20:00 23:00 02:00 05:00\n天气Image 24Image 25Image 26Image 27Image 28Image 29Image 30Image 31\n气温 30.8℃31.1℃30.4℃29.4℃27.6℃25.9℃25.7℃25.8℃\n降水 0.5mm 1.2mm 1.8mm 7.3mm 1.2mm 0.6mm 0.6mm 0.6mm\n风速 3.3m/s 3.3m/s 3.3m/s 2.9m/s 2.7m/s 3.1m/s 3.1m/s 3m/s\n风向 西北风 东北风 西北风 西北风 西北风 西北风 西北风 东北风\n气压 1009.5hPa 1009.4hPa 100

# 2、使用ReAct模式

举例1：使用网络上现成的提示词模板结构


In [28]:
from langchain import hub
from langchain.agents import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = "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"

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
# 使用LangChain Hub中的官方ReAct提示模板
prompt_template = hub.pull("hwchase17/react-chat")

# 提供记忆的实例
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
)

# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询2025年9月9日北京的天气情况"})

# 处理响应
print(result)


D:\developTools\miniconda3\envs\pyth310\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: 2025年9月9日 北京 天气情况
```[{'title': '今天白天北京晴转阴傍晚有雷阵雨需防范强对流天气 - 资讯', 'url': 'http://news.weather.com.cn/2025/09/4325408.shtml', 'content': '首页\n预报\n预警\n雷达\n云图\n天气地图\n专业产品\n资讯\n视频\n节气\n我的天空\n\n今天白天北京晴转阴傍晚有雷阵雨 需防范强对流天气\n\n2025-09-09 07:06:05\n来源：\n中国天气网\n\n字号\n\n 大\n 中\n 小\n\n分享到\n\n#### 微信扫一扫：分享\n\n微信里点“发现”，扫一下\n\n二维码便可将本文分享至朋友圈。\n\n原创未经授权不得转载\n\n中国天气网讯 今天（9月9日）白天，北京天气晴转阴，傍晚有雷阵雨出没，部分地区可能伴有大风、冰雹和短时强降水，降雨天气或不利晚高峰出行，公众需注意防范。\n\n昨天，北京晴朗天气唱主调，紫外线强烈，大部地区最高气温在30至31℃，适宜进行户外活动。\n\n今晨，北京以晴为主。（图/中国天气网王晓）\n\n北京市气象台预计，今天白天晴转阴，傍晚有雷阵雨，北转南风二三级，最高气温30℃；夜间阴转多云，南转北风一二级，最低气温21℃。\n\n明天，北京以多云天气为主，气温变化不大，中午时段体感较热，早晚较舒适。\n\n气象部门提醒，今天傍晚前后，北京将有雷阵雨，部分地区可能伴有大风、冰雹和短时强降水，建议公众关注临近预报预警信息，随身携带雨具，注意防雷避雨。 [...] 南方高温缩减北方昼夜温差加大 明起新一轮降雨过程来袭\n\n明（9月10日）起四川盆地至西北地区东部、华北南部、黄淮等地将迎来新一轮降雨过程。南方高温天气继续缩减，将所剩无几，北方多地昼夜温差逐渐加大。\n\n 中国天气网\n 2025-09-09 07:59\n 分享\n\n#### 微信扫一扫：分享\n\n微信里点“发现”，扫一下\n\n二维码便可将本文分享至朋友圈。\n\n今天白天北京晴转阴傍晚有雷阵雨 需防

In [29]:
result = agent_executor.invoke({"input":"上海的呢？"})

print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "2025年9月9日 上海 天气预报"
```[{'title': '中国上海2025年9月的天气历史中国', 'url': 'https://zh.weatherspark.com/h/m/137446/2025/9/2025%E5%B9%B49%E6%9C%88-%E4%B8%AD%E5%9B%BD%E3%80%81%E4%B8%AD%E5%9B%BD%E4%B8%8A%E6%B5%B7-%E7%9A%84%E5%8E%86%E5%8F%B2%E5%A4%A9%E6%B0%94', 'content': '60% 大部分多云   80% 阴天   100%  无重要云未检测到云晴空万里  每小时所报告的云彩覆盖，以天空的云彩覆盖百分比分类。    2025年9月中国上海的观察天气 2025年年秋季  链接  下载  比较  平均值  历史：123456789月1011122024年2023年2022年2021年    雾薄雾细雨小雨中雨大雨冻雨雨夹雪雪粒小雪中雪大雪冰雹雷雨  每小时观察到的天气，按类型着色（按严重程度的排序）。 如果存在多项报告，将显示最严重的代码。  | 白天 | 观察 | 降水 | 代码 |  ---  --- | | 9月1日周一 | Recent Thunderstorm、Thunderstorm、Thunderstorm with Heavy Rain、Thunderstorm with Rain、Recent Showers of Rain、Showers of Rain、Showers of Light Rain |  | RETS、TS、+TSRA、TSRA、RESHRA、SHRA、-SHRA | | 9月2日周二 | Recent [...] | 28日 |  | 34% | 上午11:31 | 东南偏东 | 下午9:21 | 西南偏西  | 下午4:25 | 南 | 403,402公里 | | 29日 |  | 44% | 下午12:25 | 东南 | 下午10:12 | 西南  | 下午5:19 | 南 

举例2：自定义提示词模板

In [32]:
from langchain import hub
from langchain.agents import create_react_agent
from langchain_core.prompts import ChatPromptTemplate
# 获取Tavily搜索的实例
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
os.environ['TAVILY_API_KEY'] = "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"

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
# 使用Tool
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
# 自定义提示词模板
template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}
"""
prompt_template = PromptTemplate.from_template(
    template=template,
)

# 提供记忆的实例
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
)

# 获取Agent的实例：create_react_agent()
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool]
)

# 获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
)


# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_executor.invoke({"input":"查询2025年9月9日北京的天气情况"})

# 处理响应
print(result)




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "Beijing weather September 9 2025"
```[{'title': 'Beijing weather in September 2025', 'url': 'https://www.weather25.com/asia/china/shanxi/beijing?page=month&month=September', 'content': '## The average weather in Beijing in September\n\nThe temperatures in Beijing in September are comfortable with low of 62°F and and high up to 82°F.\n\nYou can expect about 3 to 8 days of rain in Beijing during the month of September. It’s a good idea to bring along your umbrella so that you don’t get caught in poor weather.\n\nOur weather forecast can give you a great sense of what weather to expect in Beijing in September 2025. [...] If you’re planning to visit Beijing in the near future, we highly recommend that you review the 14 day weather forecast for Beijing before you arrive.\n\nTemperatures\n\n82° / 62°\n\nRainy Days\n\n3\n\nSnowy Days\n\n0\n\nDry Days\n\n27\n\nRainfall\n\n2.32\n\nin\

In [31]:
result = agent_executor.invoke({"input":"上海的呢？"})

print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "2025年9月9日 上海 天气预报"
```[{'title': '中国上海2025年9月的天气历史中国', 'url': 'https://zh.weatherspark.com/h/m/137446/2025/9/2025%E5%B9%B49%E6%9C%88-%E4%B8%AD%E5%9B%BD%E3%80%81%E4%B8%AD%E5%9B%BD%E4%B8%8A%E6%B5%B7-%E7%9A%84%E5%8E%86%E5%8F%B2%E5%A4%A9%E6%B0%94', 'content': '60% 大部分多云   80% 阴天   100%  无重要云未检测到云晴空万里  每小时所报告的云彩覆盖，以天空的云彩覆盖百分比分类。    2025年9月中国上海的观察天气 2025年年秋季  链接  下载  比较  平均值  历史：123456789月1011122024年2023年2022年2021年    雾薄雾细雨小雨中雨大雨冻雨雨夹雪雪粒小雪中雪大雪冰雹雷雨  每小时观察到的天气，按类型着色（按严重程度的排序）。 如果存在多项报告，将显示最严重的代码。  | 白天 | 观察 | 降水 | 代码 |  ---  --- | | 9月1日周一 | Recent Thunderstorm、Thunderstorm、Thunderstorm with Heavy Rain、Thunderstorm with Rain、Recent Showers of Rain、Showers of Rain、Showers of Light Rain |  | RETS、TS、+TSRA、TSRA、RESHRA、SHRA、-SHRA | | 9月2日周二 | Recent [...] | 28日 |  | 34% | 上午11:31 | 东南偏东 | 下午9:21 | 西南偏西  | 下午4:25 | 南 | 403,402公里 | | 29日 |  | 44% | 下午12:25 | 东南 | 下午10:12 | 西南  | 下午5:19 | 南 